In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer  # Assuming Bllossom's llama model uses the Hugging Face transformers library
import torch
import json

# Define our coffee kiosk class to handle orders, responses, and JSON generation
class CoffeeKiosk:
    def __init__(self):
        # Model name for Bllossom's Llama
        model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

        # Initialize the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

        # Available menu items
        self.menu_items = [
            "아메리카노", "라떼", "카푸치노", "카페모카", "바닐라라떼",
            "에스프레소", "카라멜마끼아또", "허브티", "홍차", "초콜릿라떼",
            "레몬에이드", "복숭아아이스티", "딸기스무디", "망고스무디",
            "키위주스", "토마토주스"
        ]

        # Initialize order history and defaults
        self.order_history = []
        self.default_size = "미디움"
        self.default_temperature = "핫"

    # Function to reset order history for a new session
    def reset_order(self):
        self.order_history = []

    # Function to check if a drink is available in the menu
    def is_menu_item(self, drink_name):
        return drink_name in self.menu_items

# Initialize CoffeeKiosk instance
kiosk = CoffeeKiosk()

print("Coffee Kiosk initialized and ready to take orders.")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Coffee Kiosk initialized and ready to take orders.


In [ ]:
class CoffeeKiosk:
    def __init__(self):
        # Initialize menu, order history, and defaults
        self.menu_items = [
            "아메리카노", "라떼", "카푸치노", "카페모카", "바닐라라떼",
            "에스프레소", "카라멜마끼아또", "허브티", "홍차", "초콜릿라떼",
            "레몬에이드", "복숭아아이스티", "딸기스무디", "망고스무디",
            "키위주스", "토마토주스"
        ]
        self.order_history = []
        self.default_size = "미디움"
        self.default_temperature = "핫"
        self.shorthand_mapping = {
            "아아": "아이스 아메리카노",
            "뜨아": "핫 아메리카노"
        }

    def reset_order(self):
        """Reset the order history for a new customer session."""
        self.order_history = []

    def is_menu_item(self, drink_name):
        """Check if a drink is on the menu, including shorthand names."""
        return drink_name in self.menu_items or drink_name in self.shorthand_mapping

    def suggest_alternative(self, drink_name):
        """Suggest a similar item if requested item is unavailable."""
        if drink_name == "초코라떼":
            return "초콜릿라떼"
        return None

    def add_order_item(self, drink, quantity=1, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Add a new item to the order."""
        # Map shorthand to full drink name if necessary
        drink = self.shorthand_mapping.get(drink, drink)

        # Default values if not specified
        size = size or self.default_size
        temperature = temperature or self.default_temperature
        add_ons = add_ons or []

        item = {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size,
            "add_ons": add_ons,
            "extra_shots": extra_shots
        }

        self.order_history.append(item)

        latest_item_response = f"{drink} {quantity}잔 주문되었습니다."
        full_summary = self.get_order_summary()
        response = f"{latest_item_response} {full_summary}"

        # JSON output only for the latest item
        json_output = {
            "action": "create_order",
            "order_items": [new_item]
        }
        return response, json_output

    def modify_order(self, old_drink, new_drink, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Modify an existing order item."""
        # Locate the item in order history
        for item in self.order_history:
            if item["drink"] == old_drink:
                # Update details
                item.update({
                    "drink": new_drink,
                    "temperature": temperature or item["temperature"],
                    "size": size or item["size"],
                    "add_ons": add_ons or item["add_ons"],
                    "extra_shots": extra_shots or item["extra_shots"]
                })

                response = f"주문이 {old_drink}에서 {new_drink}로 변경되었습니다. {self.get_order_summary()}"
                json_output = {
                    "action": "modify_order",
                    "old_drink": old_drink,
                    "new_drink": new_drink,
                    "temperature": item["temperature"],
                    "size": item["size"],
                    "quantity": item["quantity"],
                    "add_ons": item["add_ons"],
                    "extra_shots": item["extra_shots"]
                }
                return response, json_output

        # If not found
        return f"{old_drink}는 주문 내역에 없습니다.", None

    def get_order_summary(self):
      """Generate a comprehensive summary of the current order history."""
      if not self.order_history:
        return "현재 주문된 항목이 없습니다."

      summary = "주문하신 내용은 다음과 같습니다:"
      for item in self.order_history:
            drink_summary = (
                f"- {item['drink']} {item['quantity']}잔 "
                f"({item['temperature']}, {item['size']}"
            )
            if item['add_ons']:
                drink_summary += f", 추가 옵션: {', '.join(item['add_ons'])}"
            if item['extra_shots']:
                drink_summary += f", 샷 추가: {item['extra_shots']}회"
            drink_summary += ")"
            summary += f"\n{drink_summary}"
      return summary

    def summarize_order_history(self):
        """Provide a summary of the entire order history without adding new JSON output."""
        if not self.order_history:
           return "현재 주문된 항목이 없습니다.", None

           response = f"지금까지의 주문내역입니다:\n{self.get_order_summary()}"
           return response, None

    def process_input(self, input_text):
        """Process customer input and route to the correct functionality."""
        # Check for order completion
        if "주문 완료할게" in input_text:
            response = "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
            json_output = {
                "action": "complete_order",
                "order_items": self.order_history
            }
            self.reset_order()
            return response, json_output

        if "지금까지 뭘 주문했지" in input_text:
            response = self.get_order_summary()
            json_output = None  # No JSON output for history summary
            return response, json_output

        # Handle new order or modification based on input
        drink = None
        for item in self.menu_items + list(self.shorthand_mapping.keys()):
            if item in input_text:
                drink = self.shorthand_mapping.get(item, item)
                break

        # If drink is not found, suggest alternatives
        if not drink:
            alternative = self.suggest_alternative(input_text)
            if alternative:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다. 대신 {alternative}를 추천드립니다."
                json_output = {
                    "action": "recommend_closest_item",
                    "requested_item": input_text,
                    "recommended_item": alternative
                }
            else:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다."
                json_output = None
            return response, json_output

        # Process adding the new item to the order
        parsed_order = self.parse_order_input(input_text)
        if parsed_order:
            response, json_output = self.add_order_item(
                drink=parsed_order["drink"],
                quantity=parsed_order.get("quantity", 1),
                temperature=parsed_order.get("temperature"),
                size=parsed_order.get("size"),
                add_ons=parsed_order.get("add_ons", []),
                extra_shots=parsed_order.get("extra_shots", 0)
            )
            return response, json_output


    def parse_order_input(self, input_text):
        """Parse input text to determine drink details."""
        # Example parsing logic
        drink = None
        for item in self.menu_items:
            if item in input_text:
                drink = item
                break

        if not drink:
            return None

        quantity = 1
        if "두잔" in input_text:
            quantity = 2
        elif "세잔" in input_text:
            quantity = 3

        temperature = "아이스" if "아이스" in input_text else None
        size = "라지" if "라지" in input_text else None

        return {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size
        }


In [ ]:
# Import necessary libraries
import json

# Define our coffee kiosk class to handle orders, responses, and JSON generation
class CoffeeKiosk:
    def __init__(self):
        # Initialize menu, order history, and defaults
        self.menu_items = [
            "아메리카노", "라떼", "카푸치노", "카페모카", "바닐라라떼",
            "에스프레소", "카라멜마끼아또", "허브티", "홍차", "초콜릿라떼",
            "레몬에이드", "복숭아아이스티", "딸기스무디", "망고스무디",
            "키위주스", "토마토주스"
        ]
        self.order_history = []
        self.default_size = "미디움"
        self.default_temperature = "핫"
        self.shorthand_mapping = {
            "아아": "아이스 아메리카노",
            "뜨아": "핫 아메리카노"
        }

    def reset_order(self):
        """Reset the order history for a new customer session."""
        self.order_history = []

    def is_menu_item(self, drink_name):
        """Check if a drink is on the menu, including shorthand names."""
        return drink_name in self.menu_items or drink_name in self.shorthand_mapping

    def suggest_alternative(self, drink_name):
        """Suggest a similar item if requested item is unavailable."""
        if drink_name == "초코라떼":
            return "초콜릿라떼"
        return None

    def add_order_item(self, drink, quantity=1, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Add a new item to the order."""
        # Map shorthand to full drink name if necessary
        drink = self.shorthand_mapping.get(drink, drink)

        # Default values if not specified
        size = size or self.default_size
        temperature = temperature or self.default_temperature
        add_ons = add_ons or []

        item = {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size,
            "add_ons": add_ons,
            "extra_shots": extra_shots
        }

        self.order_history.append(item)

        latest_item_response = f"{drink} {quantity}잔 주문되었습니다."
        full_summary = self.get_order_summary()
        response = f"{latest_item_response} {full_summary}"

        # JSON output only for the latest item
        json_output = {
            "action": "create_order",
            "order_items": [item]
        }
        return response, json_output

    def modify_order(self, old_drink, new_drink, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Modify an existing order item."""
        # Locate the item in order history
        for item in self.order_history:
            if item["drink"] == old_drink:
                # Update details
                item.update({
                    "drink": new_drink,
                    "temperature": temperature or item["temperature"],
                    "size": size or item["size"],
                    "add_ons": add_ons or item["add_ons"],
                    "extra_shots": extra_shots or item["extra_shots"]
                })

                response = f"주문이 {old_drink}에서 {new_drink}로 변경되었습니다. {self.get_order_summary()}"
                json_output = {
                    "action": "modify_order",
                    "old_drink": old_drink,
                    "new_drink": new_drink,
                    "temperature": item["temperature"],
                    "size": item["size"],
                    "quantity": item["quantity"],
                    "add_ons": item["add_ons"],
                    "extra_shots": item["extra_shots"]
                }
                return response, json_output

        # If not found
        return f"{old_drink}는 주문 내역에 없습니다.", None

    def get_order_summary(self):
        """Generate a comprehensive summary of the current order history."""
        if not self.order_history:
            return "현재 주문된 항목이 없습니다."

        summary = "주문하신 내용은 다음과 같습니다:"
        for item in self.order_history:
            drink_summary = (
                f"- {item['drink']} {item['quantity']}잔 "
                f"({item['temperature']}, {item['size']}"
            )
            if item['add_ons']:
                drink_summary += f", 추가 옵션: {', '.join(item['add_ons'])}"
            if item['extra_shots']:
                drink_summary += f", 샷 추가: {item['extra_shots']}회"
            drink_summary += ")"
            summary += f"\n{drink_summary}"
        return summary

    def summarize_order_history(self):
        """Provide a summary of the entire order history without adding new JSON output."""
        if not self.order_history:
            return "현재 주문된 항목이 없습니다.", None

        response = f"지금까지의 주문내역입니다:\n{self.get_order_summary()}"
        return response, None

    def process_input(self, input_text):
        """Process customer input and route to the correct functionality."""
        # Check for order completion
        if "주문 완료할게" in input_text:
            response = "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
            json_output = {
                "action": "complete_order",
                "order_items": self.order_history
            }
            self.reset_order()
            return response, json_output

        if "지금까지 뭘 주문했지" in input_text:
            response = self.get_order_summary()
            json_output = None  # No JSON output for history summary
            return response, json_output

        # Handle new order or modification based on input
        drink = None
        for item in self.menu_items + list(self.shorthand_mapping.keys()):
            if item in input_text:
                drink = self.shorthand_mapping.get(item, item)
                break

        # If drink is not found, suggest alternatives
        if not drink:
            alternative = self.suggest_alternative(input_text)
            if alternative:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다. 대신 {alternative}를 추천드립니다."
                json_output = {
                    "action": "recommend_closest_item",
                    "requested_item": input_text,
                    "recommended_item": alternative
                }
            else:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다."
                json_output = None
            return response, json_output

        # Process adding the new item to the order
        parsed_order = self.parse_order_input(input_text)
        if parsed_order:
            response, json_output = self.add_order_item(
                drink=parsed_order["drink"],
                quantity=parsed_order.get("quantity", 1),
                temperature=parsed_order.get("temperature"),
                size=parsed_order.get("size"),
                add_ons=parsed_order.get("add_ons", []),
                extra_shots=parsed_order.get("extra_shots", 0)
            )
            return response, json_output


    def parse_order_input(self, input_text):
        """Parse input text to determine drink details."""
        # Example parsing logic
        drink = None
        for item in self.menu_items:
            if item in input_text:
                drink = item
                break

        if not drink:
            return None

        quantity = 1
        if "두잔" in input_text:
            quantity = 2
        elif "세잔" in input_text:
            quantity = 3

        temperature = "아이스" if "아이스" in input_text else None
        size = "라지" if "라지" in input_text else None

        return {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size
        }

# Initialize the coffee kiosk instance
kiosk = CoffeeKiosk()

# Interactive loop to simulate continuous conversation
print("Coffee Kiosk initialized and ready to take orders.")
while True:
    customer_input = input("Customer: ")
    response, json_output = kiosk.process_input(customer_input)
    print("Kiosk:", response)
    if json_output:
        print("JSON Output:", json.dumps(json_output, ensure_ascii=False, indent=2))
    if "주문 완료할게" in customer_input:
        break


Coffee Kiosk initialized and ready to take orders.
Customer: 아메리카노 한잔주세요
Kiosk: 아메리카노 1잔 주문되었습니다. 주문하신 내용은 다음과 같습니다:
- 아메리카노 1잔 (핫, 미디움)
JSON Output: {
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "quantity": 1,
      "temperature": "핫",
      "size": "미디움",
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
Customer:  에스프레소 두잔 시원하게 주실래요?
Kiosk: 에스프레소 2잔 주문되었습니다. 주문하신 내용은 다음과 같습니다:
- 아메리카노 1잔 (핫, 미디움)
- 에스프레소 2잔 (핫, 미디움)
JSON Output: {
  "action": "create_order",
  "order_items": [
    {
      "drink": "에스프레소",
      "quantity": 2,
      "temperature": "핫",
      "size": "미디움",
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
Customer: 핫 아메리카노 2잔이랑 아이스 아메리카노 3잔 주세요
Kiosk: 아메리카노 1잔 주문되었습니다. 주문하신 내용은 다음과 같습니다:
- 아메리카노 1잔 (핫, 미디움)
- 에스프레소 2잔 (핫, 미디움)
- 아메리카노 1잔 (아이스, 미디움)
JSON Output: {
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "quantity": 1,
      "temperature": "아이스",
      "size": "미디움",
      "a

KeyboardInterrupt: Interrupted by user

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

# Define our coffee kiosk class to handle orders, responses, and JSON generation
class CoffeeKiosk:
    def __init__(self):
        # Model name for Bllossom's Llama
        model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

        # Initialize the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

        # Available menu items
        self.menu_items = [
            "아메리카노", "라떼", "카푸치노", "카페모카", "바닐라라떼",
            "에스프레소", "카라멜마끼아또", "허브티", "홍차", "초콜릿라떼",
            "레몬에이드", "복숭아아이스티", "딸기스무디", "망고스무디",
            "키위주스", "토마토주스"
        ]
        self.order_history = []
        self.default_size = "미디움"
        self.default_temperature = "핫"
        self.shorthand_mapping = {
            "아아": "아이스 아메리카노",
            "뜨아": "핫 아메리카노"
        }

    def reset_order(self):
        """Reset the order history for a new customer session."""
        self.order_history = []

    def is_menu_item(self, drink_name):
        """Check if a drink is on the menu, including shorthand names."""
        return drink_name in self.menu_items or drink_name in self.shorthand_mapping

    def suggest_alternative(self, drink_name):
        """Suggest a similar item if requested item is unavailable."""
        if drink_name == "초코라떼":
            return "초콜릿라떼"
        return None

    def add_order_item(self, drink, quantity=1, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Add a new item to the order."""
        # Map shorthand to full drink name if necessary
        drink = self.shorthand_mapping.get(drink, drink)

        # Default values if not specified
        size = size or self.default_size
        temperature = temperature or self.default_temperature
        add_ons = add_ons or []

        item = {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size,
            "add_ons": add_ons,
            "extra_shots": extra_shots
        }

        self.order_history.append(item)

        latest_item_response = f"{drink} {quantity}잔 주문되었습니다."
        full_summary = self.get_order_summary()
        response = f"{latest_item_response} {full_summary}"

        # JSON output only for the latest item
        json_output = {
            "action": "create_order",
            "order_items": [item]
        }
        return response, json_output

    def modify_order(self, old_drink, new_drink, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Modify an existing order item."""
        # Locate the item in order history
        for item in self.order_history:
            if item["drink"] == old_drink:
                # Update details
                item.update({
                    "drink": new_drink,
                    "temperature": temperature or item["temperature"],
                    "size": size or item["size"],
                    "add_ons": add_ons or item["add_ons"],
                    "extra_shots": extra_shots or item["extra_shots"]
                })

                response = f"주문이 {old_drink}에서 {new_drink}로 변경되었습니다. {self.get_order_summary()}"
                json_output = {
                    "action": "modify_order",
                    "old_drink": old_drink,
                    "new_drink": new_drink,
                    "temperature": item["temperature"],
                    "size": item["size"],
                    "quantity": item["quantity"],
                    "add_ons": item["add_ons"],
                    "extra_shots": item["extra_shots"]
                }
                return response, json_output

        # If not found
        return f"{old_drink}는 주문 내역에 없습니다.", None

    def get_order_summary(self):
      """Generate a comprehensive summary of the current order history."""
      if not self.order_history:
        return "현재 주문된 항목이 없습니다."

      summary = "주문하신 내용은 다음과 같습니다:"
      for item in self.order_history:
            drink_summary = (
                f"- {item['drink']} {item['quantity']}잔 "
                f"({item['temperature']}, {item['size']}"
            )
            if item['add_ons']:
                drink_summary += f", 추가 옵션: {', '.join(item['add_ons'])}"
            if item['extra_shots']:
                drink_summary += f", 샷 추가: {item['extra_shots']}회"
            drink_summary += ")"
            summary += f"\n{drink_summary}"
      return summary

    def summarize_order_history(self):
        """Provide a summary of the entire order history without adding new JSON output."""
        if not self.order_history:
           return "현재 주문된 항목이 없습니다.", None

           response = f"지금까지의 주문내역입니다:\n{self.get_order_summary()}"
           return response, None

    def process_input(self, input_text):
        """Process customer input and route to the correct functionality."""
        # Check for order completion
        if "주문 완료할게" in input_text:
            response = "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
            json_output = {
                "action": "complete_order",
                "order_items": self.order_history
            }
            self.reset_order()
            return response, json_output

        if "지금까지 뭘 주문했지" in input_text:
            response = self.get_order_summary()
            json_output = None  # No JSON output for history summary
            return response, json_output

        # Handle new order or modification based on input
        drink = None
        for item in self.menu_items + list(self.shorthand_mapping.keys()):
            if item in input_text:
                drink = self.shorthand_mapping.get(item, item)
                break

        # If drink is not found, suggest alternatives
        if not drink:
            alternative = self.suggest_alternative(input_text)
            if alternative:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다. 대신 {alternative}를 추천드립니다."
                json_output = {
                    "action": "recommend_closest_item",
                    "requested_item": input_text,
                    "recommended_item": alternative
                }
            else:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다."
                json_output = None
            return response, json_output

        # Process adding the new item to the order
        parsed_order = self.parse_order_input(input_text)
        if parsed_order:
            response, json_output = self.add_order_item(
                drink=parsed_order["drink"],
                quantity=parsed_order.get("quantity", 1),
                temperature=parsed_order.get("temperature"),
                size=parsed_order.get("size"),
                add_ons=parsed_order.get("add_ons", []),
                extra_shots=parsed_order.get("extra_shots", 0)
            )
            return response, json_output


    def parse_order_input(self, input_text):
        """Parse input text to determine drink details."""
        # Example parsing logic
        drink = None
        for item in self.menu_items:
            if item in input_text:
                drink = item
                break

        if not drink:
            return None

        quantity = 1
        if "두잔" in input_text:
            quantity = 2
        elif "세잔" in input_text:
            quantity = 3

        temperature = "아이스" if "아이스" in input_text else None
        size = "라지" if "라지" in input_text else None

        return {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size
        }

    # Initialize the coffee kiosk instance
kiosk = CoffeeKiosk()

# Interactive loop to simulate continuous conversation
print("Coffee Kiosk initialized and ready to take orders.")
while True:
    customer_input = input("Customer: ")
    response, json_output = kiosk.process_input(customer_input)
    print("Kiosk:", response)
    if json_output:
        print("JSON Output:", json.dumps(json_output, ensure_ascii=False, indent=2))
    if "주문 완료할게" in customer_input:
        break


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Coffee Kiosk initialized and ready to take orders.
Customer: 카페라떼 라지로 2잔 주세요
Kiosk: 라떼 1잔 주문되었습니다. 주문하신 내용은 다음과 같습니다:
- 라떼 1잔 (핫, 라지)
JSON Output: {
  "action": "create_order",
  "order_items": [
    {
      "drink": "라떼",
      "quantity": 1,
      "temperature": "핫",
      "size": "라지",
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
Customer: 에스프레소 두잔 시원하게 주실래요?
Kiosk: 에스프레소 2잔 주문되었습니다. 주문하신 내용은 다음과 같습니다:
- 라떼 1잔 (핫, 라지)
- 에스프레소 2잔 (핫, 미디움)
JSON Output: {
  "action": "create_order",
  "order_items": [
    {
      "drink": "에스프레소",
      "quantity": 2,
      "temperature": "핫",
      "size": "미디움",
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
Customer: 아메리카노 한잔 주시고 카라멜마끼아또도 엑스라지로 한잔만 주세요
Kiosk: 아메리카노 1잔 주문되었습니다. 주문하신 내용은 다음과 같습니다:
- 라떼 1잔 (핫, 라지)
- 에스프레소 2잔 (핫, 미디움)
- 아메리카노 1잔 (핫, 라지)
JSON Output: {
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "quantity": 1,
      "temperature": "핫",
      "size": "라지",
      "add_ons": [],
   

KeyboardInterrupt: Interrupted by user

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

class CoffeeKiosk:
    def __init__(self):
        # Model setup for Llama
        model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Set device to GPU if available, otherwise CPU
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Load model and move to specified device
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)


        # Menu items and shorthand mappings
        self.menu_items = [
            "아메리카노", "라떼", "카푸치노", "카페모카", "바닐라라떼",
            "에스프레소", "카라멜마끼아또", "허브티", "홍차", "초콜릿라떼",
            "레몬에이드", "복숭아아이스티", "딸기스무디", "망고스무디",
            "키위주스", "토마토주스"
        ]
        self.order_history = []
        self.default_size = "미디움"
        self.default_temperature = "핫"
        self.shorthand_mapping = {"아아": "아이스 아메리카노", "뜨아": "핫 아메리카노"}

    def is_menu_item(self, drink_name):
        """Check if a drink is on the menu, including shorthand names."""
        return drink_name in self.menu_items or drink_name in self.shorthand_mapping

    def suggest_alternative(self, drink_name):
        """Suggest a similar item if requested item is unavailable."""
        if drink_name == "초코라떼":
            return "초콜릿라떼"
        return None

    def reset_order(self):
        """Reset the order history for a new customer session."""
        self.order_history = []

    def generate_response_with_model(self, prompt):
        # Tokenize input and move input tensor to the device
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        # Generate output from the model
        output = self.model.generate(**inputs,max_new_tokens=100)

        # Decode the output and return the generated response
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return response
    def parse_order_input(self, input_text):
        """Parse input text to determine drink details."""
        # Example parsing logic
        drink = None
        for item in self.menu_items:
            if item in input_text:
                drink = item
                break

        if not drink:
            return None

        quantity = 1
        if "두잔" in input_text:
            quantity = 2
        elif "세잔" in input_text:
            quantity = 3

        temperature = "아이스" if "아이스" in input_text else None
        size = "라지" if "라지" in input_text else None

        return {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size
        }

    def process_input(self, input_text):
        """Process customer input and route to the correct functionality."""
        # Check for order completion
        if "주문 완료할게" in input_text:
            response = "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
            json_output = {
                "action": "complete_order",
                "order_items": self.order_history
            }
            self.reset_order()
            return response, json_output

        if "지금까지 뭘 주문했지" in input_text:
            response = self.get_order_summary()
            json_output = None  # No JSON output for history summary
            return response, json_output

        # Handle new order or modification based on input
        drink = None
        for item in self.menu_items + list(self.shorthand_mapping.keys()):
            if item in input_text:
                drink = self.shorthand_mapping.get(item, item)
                break

        # If drink is not found, suggest alternatives
        if not drink:
            alternative = self.suggest_alternative(input_text)
            if alternative:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다. 대신 {alternative}를 추천드립니다."
                json_output = {
                    "action": "recommend_closest_item",
                    "requested_item": input_text,
                    "recommended_item": alternative
                }
            else:
                response = f"죄송합니다, {input_text}는 메뉴에 없습니다."
                json_output = None
            return response, json_output

        # Process adding the new item to the order
        parsed_order = self.parse_order_input(input_text)
        if parsed_order:
            response, json_output = self.add_order_item(
                drink=parsed_order["drink"],
                quantity=parsed_order.get("quantity", 1),
                temperature=parsed_order.get("temperature"),
                size=parsed_order.get("size"),
                add_ons=parsed_order.get("add_ons", []),
                extra_shots=parsed_order.get("extra_shots", 0)
            )
            return response, json_output


    def parse_llama_response(self, response_text):
        """Parse Llama-generated text to determine drink details."""
        # Extract drink, quantity, temperature, size, add-ons from the model's response
        drink = None
        for item in self.menu_items + list(self.shorthand_mapping.keys()):
            if item in response_text:
                drink = self.shorthand_mapping.get(item, item)
                break

        if not drink:
            return None

        # Parse quantity, temperature, size, add-ons based on Llama output
        quantity = 1
        if "두잔" in response_text:
            quantity = 2
        elif "세잔" in response_text:
            quantity = 3
        elif "네잔" in response_text:
            quantity = 4

        temperature = "아이스" if "아이스" in response_text else "핫"
        size = "라지" if "라지" in response_text or "엑스라지" in response_text else self.default_size
        add_ons = ["휘핑크림"] if "휘핑크림 추가" in response_text else []

        return {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size,
            "add_ons": add_ons
        }

    def add_order_item(self, drink, quantity=1, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Add a new item to the order."""
        item = {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature or self.default_temperature,
            "size": size or self.default_size,
            "add_ons": add_ons or [],
            "extra_shots": extra_shots
        }
        self.order_history.append(item)

        latest_item_response = f"{drink} {quantity}잔 주문되었습니다."
        full_summary = self.get_order_summary()
        response = f"{latest_item_response} {full_summary}"

        # JSON output only for the latest item
        json_output = {
            "action": "create_order",
            "order_items": [item]
        }
        return response, json_output
    def modify_order(self, old_drink, new_drink, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Modify an existing order item."""
        # Locate the item in order history
        for item in self.order_history:
            if item["drink"] == old_drink:
                # Update details
                item.update({
                    "drink": new_drink,
                    "temperature": temperature or item["temperature"],
                    "size": size or item["size"],
                    "add_ons": add_ons or item["add_ons"],
                    "extra_shots": extra_shots or item["extra_shots"]
                })

                response = f"주문이 {old_drink}에서 {new_drink}로 변경되었습니다. {self.get_order_summary()}"
                json_output = {
                    "action": "modify_order",
                    "old_drink": old_drink,
                    "new_drink": new_drink,
                    "temperature": item["temperature"],
                    "size": item["size"],
                    "quantity": item["quantity"],
                    "add_ons": item["add_ons"],
                    "extra_shots": item["extra_shots"]
                }
                return response, json_output

        # If not found
        return f"{old_drink}는 주문 내역에 없습니다.", None

    def get_order_summary(self):
        """Generate a comprehensive summary of the current order history."""
        if not self.order_history:
            return "현재 주문된 항목이 없습니다."

        summary = "주문하신 내용은 다음과 같습니다:"
        for item in self.order_history:
            drink_summary = (
                f"- {item['drink']} {item['quantity']}잔 "
                f"({item['temperature']}, {item['size']}"
            )
            if item['add_ons']:
                drink_summary += f", 추가 옵션: {', '.join(item['add_ons'])}"
            if item['extra_shots']:
                drink_summary += f", 샷 추가: {item['extra_shots']}회"
            drink_summary += ")"
            summary += f"\n{drink_summary}"
        return summary

    def summarize_order_history(self):
        """Provide a summary of the entire order history without adding new JSON output."""
        if not self.order_history:
           return "현재 주문된 항목이 없습니다.", None

           response = f"지금까지의 주문내역입니다:\n{self.get_order_summary()}"
           return response, None


# Initialize Coffee Kiosk
kiosk = CoffeeKiosk()

# Simulate conversation loop
print("Coffee Kiosk initialized and ready to take orders.")
while True:
    customer_input = input("Customer: ")
    response, json_output = kiosk.process_input(customer_input)
    print("Kiosk:", response)
    if json_output:
        print("JSON Output:", json.dumps(json_output, ensure_ascii=False, indent=2))
    if "주문 완료할게" in customer_input:
        break


Exception: data did not match any variant of untagged enum ModelWrapper at line 1251003 column 3

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
import re

class CoffeeKiosk:
    def __init__(self):
        # Model setup for Llama
        model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Set device to GPU if available, otherwise CPU
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Load model and move to specified device with trust_remote_code=True
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(self.device)

        # Menu items and shorthand mappings
        self.menu_items = [
            "아메리카노", "라떼", "카푸치노", "카페모카", "바닐라라떼",
            "에스프레소", "카라멜마끼아또", "허브티", "홍차", "초콜릿라떼",
            "레몬에이드", "복숭아아이스티", "딸기스무디", "망고스무디",
            "키위주스", "토마토주스"
        ]
        self.order_history = []
        self.default_size = "미디움"
        self.default_temperature = "핫"
        self.shorthand_mapping = {"아아": "아이스 아메리카노", "뜨아": "핫 아메리카노"}

    def is_menu_item(self, drink_name):
        """Check if a drink is on the menu, including shorthand names."""
        return drink_name in self.menu_items or drink_name in self.shorthand_mapping

    def suggest_alternative(self, drink_name):
        """Suggest a similar item if requested item is unavailable."""
        if drink_name == "초코라떼":
            return "초콜릿라떼"
        return None

    def reset_order(self):
        """Reset the order history for a new customer session."""
        self.order_history = []

    def generate_response_with_model(self, prompt):
        # Tokenize input and move input tensor to the device
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        # Remove 'token_type_ids' if it exists to avoid errors
        inputs.pop("token_type_ids", None)

        # Generate output from the model with a specified max_new_tokens
        output = self.model.generate(**inputs, max_new_tokens=100)

        # Decode the output and return the generated response
        response = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return response

    def parse_order_input(self, input_text):
        """Parse input text to determine drink details."""
        # Example parsing logic
        drink = None
        for item in self.menu_items:
            if item in input_text:
                drink = item
                break

        if not drink:
            return None

        quantity = 1
        if "두잔" in input_text:
            quantity = 2
        elif "세잔" in input_text:
            quantity = 3

        temperature = "아이스" if "아이스" in input_text else None
        size = "라지" if "라지" in input_text else None

        return {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size
        }

    def parse_multiple_items(self, input_text):
        """Parse multiple items in a single input for better handling."""
        # Regex pattern to match drink items with quantity, temperature, and size
        pattern = r"(아이스|핫)?\s*([가-힣]+)\s*(엑스라지|라지|미디움)?\s*(\d+)?잔?"
        items = re.findall(pattern, input_text)

        orders = []
        for temp, drink, size, qty_text in items:
            drink = self.shorthand_mapping.get(drink, drink)  # Expand shorthand if necessary
            quantity = int(qty_text) if qty_text else 1
            temperature = temp if temp else self.default_temperature
            size = size if size else self.default_size

            # Only add items that are on the menu
            if drink in self.menu_items:
                orders.append({
                    "drink": drink,
                    "quantity": quantity,
                    "temperature": temperature,
                    "size": size,
                    "add_ons": [],
                    "extra_shots": 0
                })

        return orders

    def process_input(self, input_text):
        """Process customer input by routing it through the model and parsing the response."""
        # Generate response from the Llama model
        response = self.generate_response_with_model(input_text)

        # Check if the user asked for order completion
        if "주문 완료할게" in response:
            final_response = "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
            json_output = {
                "action": "complete_order",
                "order_items": self.order_history
            }
            self.reset_order()
            return final_response, json_output

        # Check if the user asked for a summary of current orders
        if "지금까지 뭘 주문했지" in response:
            order_summary = self.get_order_summary()
            return order_summary, None

        # Parse the response for adding or modifying orders
        parsed_orders = self.parse_multiple_items(input_text)

        if parsed_orders:
            # If parsed orders are found, add each item to order history
            for parsed_order in parsed_orders:
                self.order_history.append(parsed_order)

            # Generate the response and JSON for the cumulative order
            response = self.create_order_summary_response(parsed_orders)
            json_output = {
                "action": "create_order",
                "order_items": parsed_orders
            }
            return response, json_output

        # Handle change requests, if detected in the input text
        old_drink, new_drink = self.extract_change_request(input_text)
        if old_drink and new_drink:
            response, json_output = self.modify_order(old_drink, new_drink)
            return response, json_output

        # Fallback if no specific action is determined
        return response, None

    def extract_change_request(self, input_text):
        """Identify drinks to change and new drink preferences."""
        match = re.search(r"(\S+)\s+.*?를\s+(\S+)\s+로\s+바꿔줘", input_text)
        if match:
            old_drink, new_drink = match.groups()
            return old_drink, new_drink
        return None, None

    def add_order_item(self, drink, quantity=1, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Add a new item to the order."""
        item = {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature or self.default_temperature,
            "size": size or self.default_size,
            "add_ons": add_ons or [],
            "extra_shots": extra_shots
        }
        self.order_history.append(item)

        latest_item_response = f"{drink} {quantity}잔 주문되었습니다."
        full_summary = self.get_order_summary()
        response = f"{latest_item_response} {full_summary}"

        # JSON output only for the latest item
        json_output = {
            "action": "create_order",
            "order_items": [item]
        }
        return response, json_output
    def parse_llama_response(self, response_text):
        """Parse Llama-generated text to determine drink details."""
        # Extract drink, quantity, temperature, size, add-ons from the model's response
        drink = None
        for item in self.menu_items + list(self.shorthand_mapping.keys()):
            if item in response_text:
                drink = self.shorthand_mapping.get(item, item)
                break

        if not drink:
            return None

        # Parse quantity, temperature, size, add-ons based on Llama output
        quantity = 1
        if "두잔" in response_text:
            quantity = 2
        elif "세잔" in response_text:
            quantity = 3
        elif "네잔" in response_text:
            quantity = 4

        temperature = "아이스" if "아이스" in response_text else "핫"
        size = "라지" if "라지" in response_text or "엑스라지" in response_text else self.default_size
        add_ons = ["휘핑크림"] if "휘핑크림 추가" in response_text else []

        return {
            "drink": drink,
            "quantity": quantity,
            "temperature": temperature,
            "size": size,
            "add_ons": add_ons
        }

    def modify_order(self, old_drink, new_drink, temperature=None, size=None, add_ons=None, extra_shots=0):
        """Modify an existing order item."""
        # Locate the item in order history
        for item in self.order_history:
            if item["drink"] == old_drink:
                # Update details
                item.update({
                    "drink": new_drink,
                    "temperature": temperature or item["temperature"],
                    "size": size or item["size"],
                    "add_ons": add_ons or item["add_ons"],
                    "extra_shots": extra_shots or item["extra_shots"]
                })

                response = f"주문이 {old_drink}에서 {new_drink}로 변경되었습니다. {self.get_order_summary()}"
                json_output = {
                    "action": "modify_order",
                    "old_drink": old_drink,
                    "new_drink": new_drink,
                    "temperature": item["temperature"],
                    "size": item["size"],
                    "quantity": item["quantity"],
                    "add_ons": item["add_ons"],
                    "extra_shots": item["extra_shots"]
                }
                return response, json_output

        # If not found
        return f"{old_drink}는 주문 내역에 없습니다.", None

    def get_order_summary(self):
        """Generate a comprehensive summary of the current order history."""
        if not self.order_history:
            return "현재 주문된 항목이 없습니다."

        summary = "주문하신 내용은 다음과 같습니다:"
        for item in self.order_history:
            drink_summary = (
                f"- {item['drink']} {item['quantity']}잔 "
                f"({item['temperature']}, {item['size']}"
            )
            if item['add_ons']:
                drink_summary += f", 추가 옵션: {', '.join(item['add_ons'])}"
            if item['extra_shots']:
                drink_summary += f", 샷 추가: {item['extra_shots']}회"
            drink_summary += ")"
            summary += f"\n{drink_summary}"
        return summary

    def create_order_summary_response(self, parsed_orders):
        """Create a summary response for the current and cumulative orders."""
        latest_items = ", ".join(
            [f"{item['drink']} {item['quantity']}잔 ({item['temperature']}, {item['size']})" for item in parsed_orders]
        )
        latest_item_response = f"{latest_items} 주문되었습니다."

        # Generate a full summary of all items in the order history
        full_summary = self.get_order_summary()
        response = f"{latest_item_response}\n{full_summary}"

        return response

    def summarize_order_history(self):
        """Provide a summary of the entire order history without adding new JSON output."""
        if not self.order_history:
           return "현재 주문된 항목이 없습니다.", None

           response = f"지금까지의 주문내역입니다:\n{self.get_order_summary()}"
           return response, None

# Initialize Coffee Kiosk
kiosk = CoffeeKiosk()

# Simulate conversation loop
print("Coffee Kiosk initialized and ready to take orders.")
while True:
    customer_input = input("Customer: ")
    response, json_output = kiosk.process_input(customer_input)
    print("Kiosk:", response)
    if json_output:
        print("JSON Output:", json.dumps(json_output, ensure_ascii=False, indent=2))
    if "주문 완료할게" in customer_input:
        break


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Coffee Kiosk initialized and ready to take orders.
Customer: 아이스 아메리카노 6잔 주세요


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Kiosk: 아메리카노 6잔 (아이스, 미디움) 주문되었습니다.
주문하신 내용은 다음과 같습니다:
- 아메리카노 6잔 (아이스, 미디움)
JSON Output: {
  "action": "create_order",
  "order_items": [
    {
      "drink": "아메리카노",
      "quantity": 6,
      "temperature": "아이스",
      "size": "미디움",
      "add_ons": [],
      "extra_shots": 0
    }
  ]
}
Customer: 쿠키앤크림 3잔 주세요


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Kiosk: 복숭아 아이스티 한잔 주세요! 복숭아 아이스티는 맛있는 음료로, 주로 여름철에 즐기기 좋습니다. 이 음료는 주로 복숭아의 맛을 뽐내며, 아이스와 함께 마시면 더욱 맛있게 느껴집니다. 복숭아 아이스티는 다양한 맛의 복숭아를 사용하여 다양한 hương을 즐길 수 있습니다. 또한, 복숭아 아이스티는 보다 건강한
Customer: 카페라때 한잔 주세요


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Kiosk: 카페라때 한잔 주세요. (Café au lait, please.) - "나의 아내는 한 잔을 마시기 전에 항상 한 잔 더 남겼습니다. 나는 그 아내가 나의 아내가 아니라는 것을 알았습니다. 나는 그 아내가 나의 아내가 아니라는 것을 알았습니다." - "나는 그 아내가 나의 아내가 아니라는 것을 알았습니다. 나는 그 아내가 나의 아내가 아니라는 것을


KeyboardInterrupt: Interrupted by user